<a href="https://colab.research.google.com/github/WATA-Haru/trademark_research/blob/main/trademark01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ドライブをマウント

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#content直下に保存されたモデルをコピー

In [ ]:
!cp -avi "/content/drive/MyDrive/yoshida_lab/nishika/trademark02/saved-train-models/cpt-jpo-tutorial" "/content"

'/content/drive/MyDrive/yoshida_lab/nishika/trademark02/saved-train-models/cpt-jpo-tutorial' -> '/content/cpt-jpo-tutorial'
'/content/drive/MyDrive/yoshida_lab/nishika/trademark02/saved-train-models/cpt-jpo-tutorial/requirements.txt' -> '/content/cpt-jpo-tutorial/requirements.txt'
'/content/drive/MyDrive/yoshida_lab/nishika/trademark02/saved-train-models/cpt-jpo-tutorial/README.md' -> '/content/cpt-jpo-tutorial/README.md'
'/content/drive/MyDrive/yoshida_lab/nishika/trademark02/saved-train-models/cpt-jpo-tutorial/notebooks' -> '/content/cpt-jpo-tutorial/notebooks'
'/content/drive/MyDrive/yoshida_lab/nishika/trademark02/saved-train-models/cpt-jpo-tutorial/notebooks/tutorial_check_train_data.ipynb' -> '/content/cpt-jpo-tutorial/notebooks/tutorial_check_train_data.ipynb'
'/content/drive/MyDrive/yoshida_lab/nishika/trademark02/saved-train-models/cpt-jpo-tutorial/scripts' -> '/content/cpt-jpo-tutorial/scripts'
'/content/drive/MyDrive/yoshida_lab/nishika/trademark02/saved-train-models/cpt-jpo

##05_tutorial.ipynbを試しに実行し、推論結果を確認

In [ ]:
%cd /content/cpt-jpo-tutorial/scripts

/content/cpt-jpo-tutorial/scripts


In [ ]:
!python 05_tutorial_submit.py

          gid                                           cite_gid
0  1000565813  1000209364 1000127296 1000507255 1000326557 10...
1  1000632686  1000415115 1000489070 1000084760 1000172374 10...
2  1000685875  1000190785 1000286274 1000447242 1000833216 10...
3  1000052056  1000632032 1000264167 1000428514 1000251206 10...
4  1000832828  1000831731 1000616591 1000427512 1000692307 10...


----

#05_tutorialをいじれば、画像を入力して、cite_gidが得られる関数を作成できるのではないか

##05_tutorialの解説

In [ ]:
#05_tutorial_submit.py

import pickle
import pandas as pd

 #test.csvノデータフレームを読みこむ
test_df = pd.read_csv('../data/input/test.csv')

#モデルをロード
with open(f'../data/output/similar_ids.pickle', 'rb') as f:
    sim_ids_dict = pickle.load(f)

#sub_df(submit)にtest_dfのgid(画像のアドレス)を読み込む
sub_df = test_df[['gid']].copy()

#sub_dfのcite_gidに対して、map関数（modelより）を使用
#pandas.map()の使用 -> https://note.nkmk.me/python-pandas-map-applymap-apply/
sub_df['cite_gid'] = sub_df['gid'].map(sim_ids_dict)

#isnull -> https://teratail.com/questions/119233
#shape[0]は、shape[行列の形]の0番目を取り出すもの。これが0以上だと、nullがあるから、assertで引っかかる
assert sub_df[sub_df['cite_gid'].isnull()].shape[0] == 0

#sub_dfのcite_gidに、予測結果を適用
#apply -> https://note.nkmk.me/python-pandas-map-applymap-apply/
#join -> https://note.nkmk.me/python-pandas-merge-join/
#cite_dfの値が多すぎるから、そこを20でスライスして削る
sub_df['cite_gid'] = sub_df['cite_gid'].apply(lambda x: ' '.join(map(str,x[:20])))

print(sub_df.head())

sub_df.to_csv('../data/output/test_submission.csv', index=False)

##05_tutorialを基に、get_result関数の実装

In [ ]:
#05_tutorial_submit.py

import pickle
import pandas as pd

#test.csvノデータフレームを読みこむ
test_df = pd.read_csv('../data/input/test.csv')

#モデルをロード
with open(f'../data/output/similar_ids.pickle', 'rb') as f:
    sim_ids_dict = pickle.load(f)

#sub_df(submit)にtest_dfのgid(画像のアドレス)を読み込む
sub_df = test_df[['gid']].copy()

#sub_dfのcite_gidに対して、map関数（modelより）を使用
#pandas.map()の使用 -> https://note.nkmk.me/python-pandas-map-applymap-apply/
sub_df['cite_gid'] = sub_df['gid'].map(sim_ids_dict)

#isnull -> https://teratail.com/questions/119233
#shape[0]は、shape[行列の形]の0番目を取り出すもの。これが0以上だと、nullがあるから、assertで引っかかる
assert sub_df[sub_df['cite_gid'].isnull()].shape[0] == 0

#sub_dfのcite_gidに、予測結果を適用
#apply -> https://note.nkmk.me/python-pandas-map-applymap-apply/
#join -> https://note.nkmk.me/python-pandas-merge-join/
#cite_dfの値が多すぎるから、そこを20でスライスして削る
sub_df['cite_gid'] = sub_df['cite_gid'].apply(lambda x: ' '.join(map(str,x[:20])))

print(sub_df.head())

sub_df.to_csv('../data/output/test_submission.csv', index=False)


#------------------以下、get_result関数の実装------------------
#まずは、パスは直書きでいい。最低限動くものを作ってスタート
#それができたら、次の段階としてパスを画像のアップロードで受け取り、結果を返す形にすればOK

import pickle
import pandas as pd

#１．01.png(itl画像）をinputに入れる
#ここは手作業

#モデルをロードmodel = load_model(model_path)の形になるようにする。
def load_model(model_path):
  with open(model_path, 'rb') as f:
    sim_ids_dict = pickle.load(f)
  return sim_ids_dict

#def preprocess(query_image_path):

#def get_image_embeddings(model, query_images)

def get_result(query_image_path, model, index):
  #query_image = preprocess(query_image_path) #ゆくゆく作る、現時点では省略
  #query_image_embeddings = get_image_embeddings(model,query_image)
  #query関数 -> https://note.nkmk.me/python-pandas-query/
  result = index.query(query_image_embeddings, k=20)
  return result



